In [4]:
import numpy as np
class dtree:
    """ A basic Decision Tree"""

    def __init__(self):
        """ Constructor """

    def read_data(self, filename):
        fid = open(filename, "r")
        data = []
        d = []
        for line in fid.readlines():
            d.append(line.strip())
        for d1 in d:
            data.append(d1.split(","))
        fid.close()

        self.feature_names = data[0]
        self.feature_names = self.feature_names[:-1]
        data = data[1:]
        self.classes = []
        for d in range(len(data)):
            self.classes.append(data[d][-1])
            data[d] = data[d][:-1]

        return data, self.classes, self.feature_names

    def classify(self, tree, datapoint):
        if type(tree) == type("string"):
            # a leaf is reached
            return tree
        else:
            a = list(tree.keys())[0]
            for i in range(len(self.feature_names)):
                if self.feature_names[i] == a:
                    break
            try:
                t = tree[a][datapoint[i]]
                return self.classify(t, datapoint)
            except:
                return None

    def classifyAll(self, tree, data):
        results = []
        for i in range(len(data)):
            results.append(self.classify(tree, data[i]))
        return results

    def make_tree(self, data, classes, feature_names, maxlevel=-1, level=0, forest=0):
        """ The main function, which recursively constructs the tree"""
        nData = len(data)
        nFeatures = len(data[0])

        try:
            self.feature_names
        except:
            self.feature_names = feature_names

        # List the possible classes
        new_classes = []
        for aclass in classes:
            if new_classes.count(aclass) == 0:
                new_classes.append(aclass)

        # Compute the default class (and total entropy)
        frequency = np.zeros(len(new_classes))

        totalEntropy = 0
        totalGini = 0
        index = 0
        for aclass in new_classes:
            frequency[index] = classes.count(aclass)
            totalEntropy += self.calc_entropy(float(frequency[index]) / nData)
            totalGini += (float(frequency[index]) / nData) ** 2

            index += 1

        totalGini = 1 - totalGini
        default = classes[np.argmax(frequency)]

        if nData == 0 or nFeatures == 0 or (maxlevel >= 0 and level > maxlevel):
            # Have reached an empty branch
            return default
        elif classes.count(classes[0]) == nData:
            # Only 1 class remains
            return classes[0]
        else:

            # Choose which feature is best
            gain = np.zeros(nFeatures)
            ggain = np.zeros(nFeatures)
            featureSet = list(range(nFeatures))
            if forest != 0:
                np.random.shuffle(featureSet)
                featureSet = featureSet[0:forest]
            for feature in featureSet:
                g, gg = self.calc_info_gain(data, classes, feature)
                gain[feature] = totalEntropy - g
                ggain[feature] = totalGini - gg

            bestFeature = np.argmax(gain)
            tree = {feature_names[bestFeature]: {}}

            # List the values that bestFeature can take
            values = []
            for datapoint in data:
                if datapoint[feature] not in values:
                    values.append(datapoint[bestFeature])

            for value in values:
                # Find the datapoints with each feature value
                newData = []
                new_classes = []
                index = 0
                for datapoint in data:
                    if datapoint[bestFeature] == value:
                        if bestFeature == 0:
                            newdatapoint = datapoint[1:]
                            newNames = feature_names[1:]
                        elif bestFeature == nFeatures:
                            newdatapoint = datapoint[:-1]
                            newNames = feature_names[:-1]
                        else:
                            newdatapoint = datapoint[:bestFeature]
                            newdatapoint.extend(datapoint[bestFeature + 1:])
                            newNames = feature_names[:bestFeature]
                            newNames.extend(feature_names[bestFeature + 1:])
                        newData.append(newdatapoint)
                        new_classes.append(classes[index])
                    index += 1

                # Now recurse to the next level
                subtree = self.make_tree(newData, new_classes, newNames, maxlevel, level + 1, forest)

                # And on returning, add the subtree on to the tree
                tree[feature_names[bestFeature]][value] = subtree

            return tree

    def printTree(self, tree, name):
        if type(tree) == dict:
            print(name, list(tree.keys())[0])
            for item in list(list(tree.values())[0].keys()):
                print(name, item)
                self.printTree(list(tree.values())[0][item], name + "\t")
        else:
            print
            name, "\t->\t", tree

    def calc_entropy(self, p):
        if p != 0:
            return -p * np.log2(p)
        else:
            return 0

    def calc_info_gain(self, data, classes, feature):

        # Calculates the information gain based on both entropy and the Gini impurity
        gain = 0
        ggain = 0
        nData = len(data)

        # List the values that feature can take

        values = []
        for datapoint in data:
            if datapoint[feature] not in values:
                values.append(datapoint[feature])

        featureCounts = np.zeros(len(values))
        entropy = np.zeros(len(values))
        gini = np.zeros(len(values))
        valueIndex = 0
        # Find where those values appear in data[feature] and the corresponding class
        for value in values:
            dataIndex = 0
            new_classes = []
            for datapoint in data:
                if datapoint[feature] == value:
                    featureCounts[valueIndex] += 1
                    new_classes.append(classes[dataIndex])
                dataIndex += 1

            # Get the values in new_classes
            classValues = []
            for aclass in new_classes:
                if classValues.count(aclass) == 0:
                    classValues.append(aclass)

            classCounts = np.zeros(len(classValues))
            classIndex = 0
            for classValue in classValues:
                for aclass in new_classes:
                    if aclass == classValue:
                        classCounts[classIndex] += 1
                classIndex += 1

            for classIndex in range(len(classValues)):
                entropy[valueIndex] += self.calc_entropy(float(classCounts[classIndex]) / np.sum(classCounts))
                gini[valueIndex] += (float(classCounts[classIndex]) / np.sum(classCounts)) ** 2

            # Computes both the Gini gain and the entropy
            gain = gain + float(featureCounts[valueIndex]) / nData * entropy[valueIndex]
            ggain = ggain + float(featureCounts[valueIndex]) / nData * gini[valueIndex]
            valueIndex += 1
        return gain, 1 - ggain

### Homework. Random Forest. 

Build a random forest based on dtree class. Use the same dataset cars for testing.

In [5]:
# the code from https://seat.massey.ac.nz/personal/s.r.marsland/Code/Ch13/randomforest.py

class randomforest:

    """The random forest algorithm based on the decision tree of Chapter 6"""
    def __init__(self):
        """ Constructor """
        self.tree = dtree()

        
    def rf(self, data, targets, features, nTrees, nSamples, nFeatures, maxlevel=5):
    
        nPoints = np.shape(data)[0]
        nDim = np.shape(data)[1]
        self.nSamples = nSamples
        self.nTrees = nTrees
                 
        classifiers = []
   
        for i in range(nTrees):
            print(i)
            # Compute bootstrap samples
            samplePoints = np.random.randint(0, nPoints, (nPoints, nSamples))
        
            for j in range(nSamples):
                sample = []
                sampleTarget = []
                for k in range(nPoints):
                    sample.append(data[samplePoints[k, j]])
                    sampleTarget.append(targets[samplePoints[k, j]])
            # Train classifiers
            classifiers.append(self.tree.make_tree(sample, sampleTarget, features, maxlevel, forest=nFeatures))
        return classifiers
    
    def rfclass(self, classifiers, data):
        decision = []
        # Majority voting
        for j in range(len(data)):
            outputs = []
            #print data[j]
            for i in range(self.nTrees):
                out = self.tree.classify(classifiers[i], data[j])
                if out is not None:
                    outputs.append(out)
            # List the possible outputs
            out = []
            for each in outputs:
                if out.count(each) == 0:
                    out.append(each)
            frequency = np.zeros(len(out))
        
            index = 0
            if len(out) > 0:
                for each in out:
                    frequency[index] = outputs.count(each)
                    index += 1
                decision.append(out[frequency.argmax()])
            else:
                decision.append(None)
        return decision

In [6]:
tree = dtree()

data, classes, features = tree.read_data('car.data')

train = data[::2][:]
test = data[1::2][:]
trainc = classes[::2]
testc = classes[1::2]

t = tree.make_tree(train, trainc, features)
out = tree.classifyAll(t, test)
tree.printTree(t,' ')
 
a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))
 
for i in range(len(out)):
    if testc[i] == 'good' or testc[i]== 'v-good':
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1

print("Tree")
print("Number correctly predicted", str(np.sum(a)))
print("Number of testpoints ", str(len(a)))
print("Percentage Accuracy ", str(100.0 * np.sum(a) / len(a)))
print("")
print("Number of cars rated as good or very good", str(np.sum(b)))
print("Number correctly identified as good or very good", str(np.sum(d)))
print("Percentage Accuracy", str(100.0 * np.sum(d) / np.sum(b)))

  safety
  low
  high
 	 persons
 	 2
 	 4
 		 buying
 		 vhigh
 			 maintenance
 			 vhigh
 			 high
 			 med
 			 low
 		 high
 			 maintenance
 			 vhigh
 			 high
 			 med
 			 low
 		 med
 			 maintenance
 			 vhigh
 			 high
 			 med
 				 luggage
 				 med
 					 doors
 					 2
 					 4
 				 small
 				 big
 			 low
 				 luggage
 				 med
 					 doors
 					 2
 					 4
 				 small
 				 big
 		 low
 			 maintenance
 			 vhigh
 			 high
 				 luggage
 				 med
 					 doors
 					 2
 					 4
 				 small
 				 big
 			 med
 				 luggage
 				 med
 					 doors
 					 2
 					 4
 				 small
 				 big
 			 low
 				 luggage
 				 med
 					 doors
 					 2
 					 4
 				 small
 				 big
 	 more
 		 buying
 		 vhigh
 			 maintenance
 			 vhigh
 			 high
 			 med
 				 doors
 				 2
 					 luggage
 					 small
 					 big
 				 3
 				 4
 				 5more
 			 low
 				 doors
 				 2
 					 luggage
 					 small
 					 big
 				 3
 				 4
 				 5more
 		 high
 			 maintenance
 			 vhigh
 			 h

In [7]:
rf = randomforest()

classifiers = rf.rf(train, trainc, features, 5, 100, 5, maxlevel=5)
out = rf.rfclass(classifiers, test)
 
a = np.zeros(len(out))
for i in range(len(out)):
    if out[i] == testc[i]:
        a[i] = 1 

print("-----")
print("Random Forest")
print("Number correctly predicted", str(np.sum(a)))
print("Number of testpoints ", str(len(a)))
print("Percentage Accuracy ", str(100.0 * np.sum(a) / len(a)))

0
1
2
3
4
-----
Random Forest
Number correctly predicted 797.0
Number of testpoints  864
Percentage Accuracy  92.24537037037037
